### Softmax回归的从零开始实现
就像从零开始实现线性回归一样，我们应该知道实现Softmax的细节

In [23]:
import torch
from IPython import display  # 用于在jupyter中显示图像
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [24]:
# 将展平每个图像，将它们视为长度为784的向量。因为我们的数据集有10个类别，所以网络输出维度为10

num_inputs = 784
num_outputs = 10

# 因为输入是784，每个输入对应输出的10个类别的概率，所以权重是784*10
W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)

b = torch.zeros(num_outputs, requires_grad=True)

In [25]:
# 给定一个矩阵`X`，我们可以对所有元素求和
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

### 实现softmax回归
$$\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}.$$

In [26]:
def softmax(X):
    # 结果是一个概率矩阵，其中每一行代表一个样本的概率分布，每个元素表示对应类别的概率。
    # 这样做的好处是，Softmax函数能够将原始的实数输出转换为概率，使得模型的输出更容易解释和处理，特别是在多类别分类任务中。

    X_exp = torch.exp(X)
    partition = X_exp.sum(dim=1,
                          keepdim=True)  # 计算X_exp每一行（对应一个样本）的元素和，即每个样本元素的指数之和
    return X_exp / partition  # 将每个样本的指数值除以相应的元素和，即对每个样本的概率进行归一化，这里应用了广播机制

In [28]:
# 我们将每个元素变成一个非负数。此外，依据概率原理，每一行的元素和为1。
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.3147, 0.2043, 0.1292, 0.3005, 0.0512],
         [0.1147, 0.5602, 0.0854, 0.0990, 0.1407]]),
 tensor([1., 1.]))

In [66]:
# 实现softmax回归模型
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [67]:
# 创建一个数据`y_hat`，其中包含2个样本在3个类别的预测概率，使用`y`作为`y_hat`中概率的索引
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

In [68]:
# 实现交叉熵损失函数
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])


cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [94]:
# 将预测类别和真实`y`元素进行比较
def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        # 这个条件判断用于检查y_hat是否为多类别的预测概率矩阵。
        # 如果y_hat的维度大于1，并且第二维度的大小大于1，说明y_hat是一个多类别的预测概率矩阵。

        y_hat = y_hat.argmax(axis=1)  # 返回最大值的索引，即预测的类别
    cmp = y_hat.type(y.dtype) == y  # 返回布尔类型的数组
    return float(cmp.type(y.dtype).sum())  # 返回正确的数量


accuracy(y_hat, y) / len(y)  # 计算正确率

0.5

In [98]:
# 我们可以评估在任意模型`net`上的准确率
def evaluate_accuracy(net, data_iter):
    """计算在指定数据集上模型的精度。

    Args:
        net (torch.nn.Module): 要评估的模型，必须是继承自torch.nn.Module的PyTorch模型。
        data_iter (torch.utils.data.DataLoader): 数据加载器，用于加载测试数据集。

    Returns:
        float: 在指定数据集上的准确率。

    Note:
        - 此函数用于评估训练好的模型在指定数据集上的性能。
        - 在评估过程中，模型将被设置为评估模式，确保测试过程中不启用Dropout等随机操作，保持一致性。
        - 使用`torch.utils.data.DataLoader`加载数据，可支持自动批量加载和数据预处理。
        - 通过调用`accuracy`函数计算每个批次的预测准确率，并使用`Accumulator`类累加结果。

    Example:
        >>> net = MyModel()
        >>> test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)
        >>> accuracy = evaluate_accuracy(net, test_data_loader)
    """

    # 如果模型是torch.nn.Module的实例，则将其设置为评估模式
    if isinstance(net, torch.nn.Module):
        net.eval()

    # 创建一个Accumulator对象用于累加正确预测的数量和总预测的数量
    metric = Accumulator(2)

    # 遍历数据加载器中的所有数据批次，X是输入特征，y是真实标签
    for X, y in data_iter:
        # 通过模型预测得到预测结果，计算当前批次的预测准确率，并累加结果
        metric.add(accuracy(net(X), y), y.numel())

    # 计算总体的准确率，即预测正确的样本数量除以总样本数量
    return metric[0] / metric[1]
